In [5]:
import pandas as pd

In [6]:
origin_india_df = pd.read_csv('../../data/amazon.csv')
# india_df = pd.read_csv('../../data/origin_in_amz_products.csv')
# canada_df = pd.read_csv('../../data/origin_ca_amz_products.csv')
# uk_df = pd.read_csv('../../data/origin_uk_amz_products.csv')
# usa_df = pd.read_csv('../../data/origin_usa_amz_products.csv')
# usa_category = pd.read_csv('../../data/origin_usa_amz_categories.csv')

In [7]:
origin_india_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1465 non-null   object
 1   product_name         1465 non-null   object
 2   category             1465 non-null   object
 3   discounted_price     1465 non-null   object
 4   actual_price         1465 non-null   object
 5   discount_percentage  1465 non-null   object
 6   rating               1465 non-null   object
 7   rating_count         1463 non-null   object
 8   about_product        1465 non-null   object
 9   user_id              1465 non-null   object
 10  user_name            1465 non-null   object
 11  review_id            1465 non-null   object
 12  review_title         1465 non-null   object
 13  review_content       1465 non-null   object
 14  img_link             1465 non-null   object
 15  product_link         1465 non-null   object
dtypes: obj

---
### India Data

In [58]:
india_df = pd.read_csv('../../data/origin_in_amz_products.csv')

In [59]:
india_df.head(2)

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,categoryName,isBestSeller,boughtInLastMonth
0,B08VJFZQ9S,प्लेन कैज़ुअल वियर बेसबॉल कैप पुरुषों और महिला...,https://m.media-amazon.com/images/I/61DK1GchGF...,https://www.amazon.in/dp/B08VJFZQ9S,0.0,0,299.0,499.0,पुरुषों के हैट्स और कैप्स,False,0
1,B08VJFYW5Q,"यूनीसेक्स कॉटन एडजस्टेबल बेसबॉल कैप (काला, फ़्...",https://m.media-amazon.com/images/I/61+nwjWLUg...,https://www.amazon.in/dp/B08VJFYW5Q,0.0,0,299.0,499.0,पुरुषों के हैट्स और कैप्स,False,0


In [60]:
india_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1589160 entries, 0 to 1589159
Data columns (total 11 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   asin               1589160 non-null  object 
 1   title              1589160 non-null  object 
 2   imgUrl             1589160 non-null  object 
 3   productURL         1589160 non-null  object 
 4   stars              1589160 non-null  float64
 5   reviews            1589160 non-null  int64  
 6   price              1589160 non-null  float64
 7   listPrice          1589160 non-null  float64
 8   categoryName       1589160 non-null  object 
 9   isBestSeller       1589160 non-null  bool   
 10  boughtInLastMonth  1589160 non-null  int64  
dtypes: bool(1), float64(3), int64(2), object(5)
memory usage: 122.8+ MB


- column name 변경 -> origin_india_df 기준으로
- reviews, isBestSeller, boughtInLastMonth 컬럼 유지
- title, categoryName 번역 필요
- discount_percentage 컬럼 추가
- 해당 데이터의 통화(INR)를 한화(KRW)로 변경 필요

In [61]:
origin_india_df.columns.unique()

Index(['product_id', 'product_name', 'category', 'discounted_price',
       'actual_price', 'discount_percentage', 'rating', 'rating_count',
       'about_product', 'user_id', 'user_name', 'review_id', 'review_title',
       'review_content', 'img_link', 'product_link'],
      dtype='object')

In [62]:
# column rename
cols = ['product_id', 'product_name', 'img_link', 'product_link', 'rating', 'reviews', 'discounted_price', 'actual_price', 'category', 'isBestSeller', 'boughtInLastMonth']
india_df.columns = cols

In [70]:
index_to_drop = india_df[(india_df['actual_price'] == 0) & (india_df['discounted_price'] == 0)].index

india_df = india_df.drop(index_to_drop)

Index([], dtype='int64')

In [71]:
# 할인율 계산
def calculate_discount_percentage(row):
    original_price = row['actual_price']
    discounted_price = row['discounted_price']
    
    if original_price <= 0 or discounted_price <= 0:
        return 0
        
    discount_percentage = ((original_price - discounted_price) / original_price) * 100
    return round(discount_percentage)

india_df['discount_percentage'] = india_df.apply(calculate_discount_percentage, axis=1)

In [72]:
# column 순서 변경
cols_num = ['product_id', 'product_name', 'category', 'discounted_price', 'actual_price', 'discount_percentage', 'rating', 'reviews', 'img_link', 'product_link', 'isBestSeller', 'boughtInLastMonth']
india_df = india_df[cols_num]

In [73]:
# 번역 모듈, 함수
from googletrans import Translator

translator = Translator()

def translate_text(text):
    return translator.translate(text, src='hi', dest='en').text

In [74]:
unique_product_names = india_df['product_name'].unique()
unique_categories = india_df['category'].unique()
print(len(unique_product_names), len(unique_categories))

1338058 214


In [75]:
# 카테고리 번역
translated_categories = {category: translate_text(category) for category in unique_categories}
india_df['category'] = india_df['category'].map(translated_categories)

In [79]:
# 제품 이름 번역 / 가짓수가 너무 많아서 번역기 오류가 남. 다른 방법을 찾아볼 것
# india_df['product_name'] = india_df['product_name'].apply(translate_text)

In [80]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('../..')))

import prep_functions

fetcher = prep_functions.ExchangeRateFetcher('KRW')

# dataframe에 적용할 함수
def convert_currency(row, col):
    if row[col] == 0:
        return 0
    return fetcher.convert(row[col], 'INR', 'KRW')

india_df['discounted_price_KRW'] = india_df.apply(lambda row: convert_currency(row, 'discounted_price'), axis=1).astype(int)
india_df['actual_price_KRW'] = india_df.apply(lambda row: convert_currency(row, 'actual_price'), axis=1).astype(int)

In [81]:
india_df.head(2)

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,reviews,img_link,product_link,isBestSeller,boughtInLastMonth,discounted_price_KRW,actual_price_KRW
0,B08VJFZQ9S,प्लेन कैज़ुअल वियर बेसबॉल कैप पुरुषों और महिला...,Men's hats and caps,299.0,499.0,40,0.0,0,https://m.media-amazon.com/images/I/61DK1GchGF...,https://www.amazon.in/dp/B08VJFZQ9S,False,0,4838,8074
1,B08VJFYW5Q,"यूनीसेक्स कॉटन एडजस्टेबल बेसबॉल कैप (काला, फ़्...",Men's hats and caps,299.0,499.0,40,0.0,0,https://m.media-amazon.com/images/I/61+nwjWLUg...,https://www.amazon.in/dp/B08VJFYW5Q,False,0,4838,8074


In [82]:
india_df.to_csv('../../data/amazon_india.csv', index=False)

---
### Canada Data

In [83]:
canada_df = pd.read_csv('../../data/origin_ca_amz_products.csv')

In [84]:
canada_df.head(2)

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,categoryName,isBestSeller,boughtInLastMonth
0,B07CV4L6HX,"Green Leaf WW3D Wonder Extension Cord Winder, ...",https://m.media-amazon.com/images/I/81cRe0AVC4...,https://www.amazon.ca/dp/B07CV4L6HX,4.4,2876,47.69,0.0,Industrial Scientific,False,0
1,B09N1HGY74,8pcs Toilet Seat Bumpers Universal Toilet Repl...,https://m.media-amazon.com/images/I/512CpB-EP5...,https://www.amazon.ca/dp/B09N1HGY74,3.8,55,10.99,0.0,Industrial Scientific,False,100


In [85]:
canada_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2165926 entries, 0 to 2165925
Data columns (total 11 columns):
 #   Column             Dtype  
---  ------             -----  
 0   asin               object 
 1   title              object 
 2   imgUrl             object 
 3   productURL         object 
 4   stars              float64
 5   reviews            int64  
 6   price              float64
 7   listPrice          float64
 8   categoryName       object 
 9   isBestSeller       bool   
 10  boughtInLastMonth  int64  
dtypes: bool(1), float64(3), int64(2), object(5)
memory usage: 167.3+ MB


- column name 변경 -> origin_india_df 기준으로
- reviews, isBestSeller, boughtInLastMonth 컬럼 유지
- discount_percentage 컬럼 추가
- 해당 데이터의 통화(CAD)를 한화(KRW)로 변경 필요

In [86]:
# india와 컬럼이 같음. column rename
# print(cols)
canada_df.columns = cols

In [87]:
index_to_drop = canada_df[(canada_df['actual_price'] == 0) & (canada_df['discounted_price'] == 0)].index

canada_df = canada_df.drop(index_to_drop)

In [88]:
# 할인율 계산
canada_df['discount_percentage'] = canada_df.apply(calculate_discount_percentage, axis=1)

In [89]:
# column 순서 변경
canada_df = canada_df[cols_num]

In [90]:
# dataframe에 적용할 함수
def convert_currency(row, col):
    if row[col] == 0:
        return 0
    return fetcher.convert(row[col], 'CAD', 'KRW')

canada_df['discounted_price_KRW'] = canada_df.apply(lambda row: convert_currency(row, 'discounted_price'), axis=1).astype(int)
canada_df['actual_price_KRW'] = canada_df.apply(lambda row: convert_currency(row, 'actual_price'), axis=1).astype(int)

In [91]:
canada_df.head(5)

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,reviews,img_link,product_link,isBestSeller,boughtInLastMonth,discounted_price_KRW,actual_price_KRW
0,B07CV4L6HX,"Green Leaf WW3D Wonder Extension Cord Winder, ...",Industrial Scientific,47.69,0.00,0,4.4,2876,https://m.media-amazon.com/images/I/81cRe0AVC4...,https://www.amazon.ca/dp/B07CV4L6HX,False,0,47217,0
1,B09N1HGY74,8pcs Toilet Seat Bumpers Universal Toilet Repl...,Industrial Scientific,10.99,0.00,0,3.8,55,https://m.media-amazon.com/images/I/512CpB-EP5...,https://www.amazon.ca/dp/B09N1HGY74,False,100,10881,0
2,B087P7538J,YaeCCC 19 Pcs Hole Saw Kit 3/4''(19mm)- 6''(15...,Industrial Scientific,25.99,27.99,7,4.0,126,https://m.media-amazon.com/images/I/71P+qLFDFc...,https://www.amazon.ca/dp/B087P7538J,False,50,25732,27712
3,B0822FF7YQ,LLPT Butyl Putty Tape White 1 Inch x 33Ft Tigh...,Industrial Scientific,21.99,30.99,29,4.5,1936,https://m.media-amazon.com/images/I/61wfWLZRn-...,https://www.amazon.ca/dp/B0822FF7YQ,False,100,21772,30683
4,B08NTR8NN4,"Lightbeam 16"" Long Stem Deep Fry Thermometer w...",Industrial Scientific,18.99,0.00,0,4.2,46,https://m.media-amazon.com/images/I/61qJG-y1He...,https://www.amazon.ca/dp/B08NTR8NN4,False,100,18801,0


In [92]:
canada_df.to_csv('../../data/amazon_canada.csv', index=False)

---
### UK Data

In [93]:
uk_df = pd.read_csv('../../data/origin_uk_amz_products.csv')

In [94]:
uk_df.head(2)

,asin,title,imgUrl,productURL,stars,reviews,price,isBestSeller,boughtInLastMonth,categoryName
0,B09B96TG33,"Echo Dot (5th generation, 2022 release) | Big ...",https://m.media-amazon.com/images/I/71C3lbbeLs...,https://www.amazon.co.uk/dp/B09B96TG33,4.7,15308,21.99,False,0,Hi-Fi Speakers
1,B01HTH3C8S,"Anker Soundcore mini, Super-Portable Bluetooth...",https://m.media-amazon.com/images/I/61c5rSxwP0...,https://www.amazon.co.uk/dp/B01HTH3C8S,4.7,98099,23.99,True,0,Hi-Fi Speakers


In [95]:
uk_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2222742 entries, 0 to 2222741
Data columns (total 10 columns):
 #   Column             Dtype  
---  ------             -----  
 0   asin               object 
 1   title              object 
 2   imgUrl             object 
 3   productURL         object 
 4   stars              float64
 5   reviews            int64  
 6   price              float64
 7   isBestSeller       bool   
 8   boughtInLastMonth  int64  
 9   categoryName       object 
dtypes: bool(1), float64(2), int64(2), object(5)
memory usage: 154.7+ MB


- india, canada와 달리 listPrice, 정가가 나와있지 않음
- column name 변경 -> origin_india_df 기준으로
- reviews, isBestSeller, boughtInLastMonth 컬럼 유지
- 해당 데이터의 통화(GBP)를 한화(KRW)로 변경 필요

In [96]:
cols.pop(7)

'actual_price'

In [97]:
uk_df.columns = cols

In [98]:
cols_num.pop(4)
cols_num.pop(4)

'discount_percentage'

In [99]:
uk_df = uk_df[cols_num]

In [101]:
index_to_drop = uk_df[(uk_df['discounted_price'] == 0)].index

uk_df = uk_df.drop(index_to_drop)

In [102]:
# dataframe에 적용할 함수
def convert_currency(row, col):
    if row[col] == 0:
        return 0
    return fetcher.convert(row[col], 'GBP', 'KRW')

uk_df['discounted_price_KRW'] = uk_df.apply(lambda row: convert_currency(row, 'discounted_price'), axis=1).astype(int)
# uk_df['actual_price_KRW'] = uk_df.apply(lambda row: convert_currency(row, 'actual_price'), axis=1).astype(int)

In [103]:
uk_df.head(2)

,product_id,product_name,category,discounted_price,rating,reviews,img_link,product_link,isBestSeller,boughtInLastMonth,discounted_price_KRW
0,B09B96TG33,"Echo Dot (5th generation, 2022 release) | Big ...",False,21.99,4.7,15308,https://m.media-amazon.com/images/I/71C3lbbeLs...,https://www.amazon.co.uk/dp/B09B96TG33,0,Hi-Fi Speakers,39128
1,B01HTH3C8S,"Anker Soundcore mini, Super-Portable Bluetooth...",True,23.99,4.7,98099,https://m.media-amazon.com/images/I/61c5rSxwP0...,https://www.amazon.co.uk/dp/B01HTH3C8S,0,Hi-Fi Speakers,42686


In [104]:
uk_df.to_csv('../../data/amazon_uk.csv', index=False)

---
### USA Data

In [105]:
usa_df = pd.read_csv('../../data/origin_usa_amz_products.csv')
usa_category = pd.read_csv('../../data/origin_usa_amz_categories.csv')

In [106]:
usa_df.head(2)

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.5,0,139.99,0.00,104,False,2000
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.5,0,169.99,209.99,104,False,1000


In [107]:
usa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1426337 entries, 0 to 1426336
Data columns (total 11 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   asin               1426337 non-null  object 
 1   title              1426336 non-null  object 
 2   imgUrl             1426337 non-null  object 
 3   productURL         1426337 non-null  object 
 4   stars              1426337 non-null  float64
 5   reviews            1426337 non-null  int64  
 6   price              1426337 non-null  float64
 7   listPrice          1426337 non-null  float64
 8   category_id        1426337 non-null  int64  
 9   isBestSeller       1426337 non-null  bool   
 10  boughtInLastMonth  1426337 non-null  int64  
dtypes: bool(1), float64(3), int64(3), object(4)
memory usage: 110.2+ MB


In [108]:
usa_category.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248 entries, 0 to 247
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             248 non-null    int64 
 1   category_name  248 non-null    object
dtypes: int64(1), object(1)
memory usage: 4.0+ KB


- product와 category가 나뉘어져있다. 매핑 필요
- column name 변경 -> origin_india_df 기준으로
- reviews, isBestSeller, boughtInLastMonth 컬럼 유지
- discount_percentage 컬럼 추가
- 해당 데이터의 통화(USD)를 한화(KRW)로 변경 필요

In [109]:
# category 정보를 전부 가지고 있는지 체크
usa_df[~usa_df['category_id'].isin(usa_category['id'])]

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth


In [110]:
# usa_category와 usa_df를 category_id와 id를 기준으로 병합
usa_df = usa_df.merge(usa_category[['id', 'category_name']], left_on='category_id', right_on='id', how='left')
usa_df.head(2)

,asin,title,imgUrl,productURL,stars,reviews,price,listPrice,category_id,isBestSeller,boughtInLastMonth,id,category_name
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,4.5,0,139.99,0.00,104,False,2000,104,Suitcases
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,4.5,0,169.99,209.99,104,False,1000,104,Suitcases


In [111]:
usa_df.drop(columns=['id', 'category_id'], inplace=True)

In [112]:
cols = ['product_id', 'product_name', 'img_link', 'product_link', 'rating', 'reviews', 'discounted_price', 'actual_price', 'isBestSeller', 'boughtInLastMonth', 'category']
cols_num = ['product_id', 'product_name', 'category', 'discounted_price', 'actual_price', 'discount_percentage', 'rating', 'reviews', 'img_link', 'product_link', 'isBestSeller', 'boughtInLastMonth']

In [113]:
usa_df.columns = cols

In [114]:
index_to_drop = usa_df[(usa_df['actual_price'] == 0) & (usa_df['discounted_price'] == 0)].index

usa_df = usa_df.drop(index_to_drop)

In [115]:
# 할인율 계산
usa_df['discount_percentage'] = usa_df.apply(calculate_discount_percentage, axis=1)

In [116]:
usa_df = usa_df[cols_num]

In [117]:
# dataframe에 적용할 함수
def convert_currency(row, col):
    if row[col] == 0:
        return 0
    return fetcher.convert(row[col], 'USD', 'KRW')

usa_df['discounted_price_KRW'] = usa_df.apply(lambda row: convert_currency(row, 'discounted_price'), axis=1).astype(int)
usa_df['actual_price_KRW'] = usa_df.apply(lambda row: convert_currency(row, 'actual_price'), axis=1).astype(int)

In [118]:
usa_df.head(2)

,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,reviews,img_link,product_link,isBestSeller,boughtInLastMonth,discounted_price_KRW,actual_price_KRW
0,B014TMV5YE,"Sion Softside Expandable Roller Luggage, Black...",Suitcases,139.99,0.00,0,4.5,0,https://m.media-amazon.com/images/I/815dLQKYIY...,https://www.amazon.com/dp/B014TMV5YE,False,2000,190722,0
1,B07GDLCQXV,Luggage Sets Expandable PC+ABS Durable Suitcas...,Suitcases,169.99,209.99,19,4.5,0,https://m.media-amazon.com/images/I/81bQlm7vf6...,https://www.amazon.com/dp/B07GDLCQXV,False,1000,231594,286089


In [119]:
usa_df.to_csv('../../data/amazon_usa.csv', index=False)